In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException

import pandas as pd
import time



In [2]:
# 신한라이프 공시실 상품공시 자료중 판매중 상품 크롤링
def scrape_shinhan_life_products_on_sale(driver):
    data0 = []

    try:
        # 1. 페이지 접속
        driver.get("https://www.shinhanlife.co.kr/hp/cdhi0030.do")
        print("공시실 상품공시 판매중 웹사이트에 접속했습니다.")
        time.sleep(3)  # 초기 페이지 로딩 대기

        current_page = 1
        while True:
            print(f"\n현재 페이지: {current_page}")
            
            wait = WebDriverWait(driver, 30)
            
            # 테이블이 제대로 로드될 때까지 대기
            max_retries = 3
            for retry in range(max_retries):
                table = wait.until(EC.presence_of_element_located((By.TAG_NAME, 'table')))
                rows = table.find_elements(By.TAG_NAME, 'tr')
                if len(rows) > 1:  # 헤더를 제외한 행이 있는지 확인
                    break
                time.sleep(2)
                if retry == max_retries - 1:
                    print("테이블 로딩 실패")
                    return pd.DataFrame(), driver

            print(f"총 {len(rows)}개의 행을 찾았습니다.")

            # URL 추출 및 변환 함수
            def get_url(element, xpath):
                elements = element.find_elements(By.XPATH, xpath)
                if elements:
                    url = elements[0].get_attribute('data-url')
                    if url:
                        if url.startswith('/repo/DigitalPlattform/'):
                            url = '/bizxpress/' + url[23:]
                        full_url = f"https://www.shinhanlife.co.kr{url}"
                        return full_url
                return 'X'

            last_category = ""
            # 각 행의 컬럼 내용을 분리하여 읽어들이기
            for index, row in enumerate(rows[1:], start=1):
                try:
                    cells = row.find_elements(By.TAG_NAME, 'td')
                    if not cells:
                        continue

                    category = cells[0].text.strip() or last_category
                    last_category = category
                    
                    product_info = cells[1].find_element(By.CLASS_NAME, 'accoHead')
                    
                    # 상품명 처리
                    product_name = product_info.find_element(By.CLASS_NAME, 'cell_1').text.strip().split('\n')[0]
                    
                    # accoHead의 URL 정보 가져오기
                    head_summary = get_url(product_info, ".//button[contains(@data-title, '요약서')]")
                    head_method = get_url(product_info, ".//button[contains(@data-title, '방법서')]")
                    head_terms = get_url(product_info, ".//button[contains(@data-title, '약관')]")
                    
                    # accoHead의 판매기간
                    head_period = product_info.find_element(By.CLASS_NAME, 'cell_2').text.strip()
                    if '~' in head_period:
                        data0.append([
                            "판매중",
                            "신한라이프",
                            category,
                            product_name,
                            head_period,
                            head_summary,
                            head_method,
                            head_terms
                        ])

                    # 상세내용 처리
                    try:
                        detail_button = product_info.find_element(By.CLASS_NAME, 'accoBtn')
                        if detail_button.is_displayed() and detail_button.is_enabled():
                            if not detail_button.get_attribute('aria-expanded') == 'true':
                                driver.execute_script("arguments[0].click();", detail_button)
                                time.sleep(0.5)

                            wait_body = WebDriverWait(driver, 3)
                            acco_body = wait_body.until(EC.presence_of_element_located((By.CLASS_NAME, 'accoBody')))
                            body_rows = acco_body.find_elements(By.CLASS_NAME, 'row')

                            for body_row in body_rows:
                                period = body_row.find_element(By.CLASS_NAME, 'cell_2').text.strip()
                                if '~' in period:
                                    body_method = get_url(body_row, ".//button[contains(@data-title, '방법서')]")
                                    body_terms = get_url(body_row, ".//button[contains(@data-title, '약관')]")
                                    
                                    data0.append([
                                        "판매중",
                                        "신한라이프",
                                        category,
                                        product_name,
                                        period,
                                        'X',
                                        body_method,
                                        body_terms
                                    ])

                            # 상세내용 닫기
                            if detail_button.get_attribute('aria-expanded') == 'true':
                                driver.execute_script("arguments[0].click();", detail_button)
                                time.sleep(0.5)
                    except Exception as e:
                        continue

                except Exception as e:
                    print(f"행 {index} 처리 중 오류 발생: {e}")
                    continue

            print(f"현재까지 총 {len(data0)}개의 행을 수집했습니다.")

            # 페이지 이동 처리
            try:
                pagination = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'paging')))
                page_links = pagination.find_elements(By.XPATH, './/ul/li[@class=""]/a')
                
                if not page_links:  # 페이지 링크가 없으면 다음 버튼 확인
                    next_button = pagination.find_element(By.CLASS_NAME, 'icoBtn_next')
                    if next_button.get_attribute('disabled') is None:
                        print("다음 페이지 세트로 이동합니다.")
                        driver.execute_script("arguments[0].click();", next_button)
                        current_page = 1
                        time.sleep(3)
                        continue
                    else:
                        print("모든 페이지의 크롤링을 완료했습니다.")
                        break

                max_pages = len(page_links) + 1
                print(f"총 {max_pages}개의 페이지가 있습니다.")
                
                if current_page < max_pages:
                    next_page_link = page_links[current_page - 1]
                    print(f"{current_page + 1}페이지로 이동합니다.")
                    driver.execute_script("arguments[0].scrollIntoView(true);", next_page_link)
                    time.sleep(1)
                    driver.execute_script("arguments[0].click();", next_page_link)
                    current_page += 1
                    time.sleep(3)
                else:
                    print("모든 페이지의 크롤링을 완료했습니다.")
                    break

            except Exception as e:
                print(f"페이지 이동 중 오류 발생: {e}")
                break

    except Exception as e:
        print(f"오류가 발생했습니다: {e}")

    df1 = pd.DataFrame(data0, columns=["판매구분", "판매사", "분류", "상품명", "판매기간", "요약서", "방법서", "약관"])
    return df1, driver

In [3]:
# 신한라이프 공시실 상품공시 자료중 판매중지 상품 크롤링
def scrape_shinhan_life_products_discontinued(driver):
    data1 = []

    try:
        # 1. 페이지 접속
        driver.get("https://www.shinhanlife.co.kr/hp/cdhi0040t01.do")
        print("\n공시실 상품공시 판매중지 웹사이트에 접속했습니다.")
        time.sleep(3)  # 초기 페이지 로딩 대기

        # URL 추출 및 변환 함수
        def get_url(element, xpath):
            elements = element.find_elements(By.XPATH, xpath)
            if elements:
                url = elements[0].get_attribute('data-url')
                if url:
                    if url.startswith('/repo/DigitalPlattform/'):
                        url = '/bizxpress/' + url[23:]
                    full_url = f"https://www.shinhanlife.co.kr{url}"
                    return full_url
            return 'X'

        # 탭 목록 가져오기
        wait = WebDriverWait(driver, 30)
        tabs = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#tabList li")))
        print(f"총 {len(tabs)}개의 탭을 찾았습니다.")

        for tab_index, tab in enumerate(tabs):
            print(f"\n{tab.text} 탭 처리 중...")
            
            # 탭 클릭
            driver.execute_script("arguments[0].click();", tab)
            time.sleep(3)  # 탭 전환 대기

            current_page = 1
            while True:
                print(f"\n현재 페이지: {current_page}")
                
                # 테이블이 제대로 로드될 때까지 대기
                max_retries = 3
                for retry in range(max_retries):
                    table = wait.until(EC.presence_of_element_located((By.TAG_NAME, 'table')))
                    rows = table.find_elements(By.TAG_NAME, 'tr')
                    if len(rows) > 1:  # 헤더를 제외한 행이 있는지 확인
                        break
                    time.sleep(2)
                    if retry == max_retries - 1:
                        print("테이블 로딩 실패")
                        continue

                print(f"총 {len(rows)}개의 행을 찾았습니다.")

                last_category = ""
                # 각 행의 컬럼 내용을 분리하여 읽어들이기
                for index, row in enumerate(rows[1:], start=1):
                    try:
                        cells = row.find_elements(By.TAG_NAME, 'td')
                        if not cells:
                            continue

                        category = cells[0].text.strip() or last_category
                        last_category = category
                        
                        product_info = cells[1].find_element(By.CLASS_NAME, 'accoHead')
                        
                        # 상품명 처리
                        product_name = product_info.find_element(By.CLASS_NAME, 'cell_1').text.strip().split('\n')[0]
                        
                        # accoHead의 URL 정보 가져오기
                        head_summary = get_url(product_info, ".//button[contains(@data-title, '요약서')]")
                        head_method = get_url(product_info, ".//button[contains(@data-title, '방법서')]")
                        head_terms = get_url(product_info, ".//button[contains(@data-title, '약관')]")
                        
                        # accoHead의 판매기간
                        head_period = product_info.find_element(By.CLASS_NAME, 'cell_2').text.strip()
                        if '~' in head_period:
                            data1.append([
                                "판매중지",
                                tab.text,
                                category,
                                product_name,
                                head_period,
                                head_summary,
                                head_method,
                                head_terms
                            ])

                        # 상세내용 처리
                        try:
                            detail_button = product_info.find_element(By.CLASS_NAME, 'accoBtn')
                            if detail_button.is_displayed() and detail_button.is_enabled():
                                if not detail_button.get_attribute('aria-expanded') == 'true':
                                    driver.execute_script("arguments[0].click();", detail_button)
                                    time.sleep(0.5)

                                wait_body = WebDriverWait(driver, 3)
                                acco_body = wait_body.until(EC.presence_of_element_located((By.CLASS_NAME, 'accoBody')))
                                body_rows = acco_body.find_elements(By.CLASS_NAME, 'row')

                                for body_row in body_rows:
                                    period = body_row.find_element(By.CLASS_NAME, 'cell_2').text.strip()
                                    if '~' in period:
                                        body_method = get_url(body_row, ".//button[contains(@data-title, '방법서')]")
                                        body_terms = get_url(body_row, ".//button[contains(@data-title, '약관')]")
                                        
                                        data1.append([
                                            "판매중지",
                                            tab.text,
                                            category,
                                            product_name,
                                            period,
                                            'X',
                                            body_method,
                                            body_terms
                                        ])

                                # 상세내용 닫기
                                if detail_button.get_attribute('aria-expanded') == 'true':
                                    driver.execute_script("arguments[0].click();", detail_button)
                                    time.sleep(0.5)
                        except Exception as e:
                            continue

                    except Exception as e:
                        print(f"행 {index} 처리 중 오류 발생: {e}")
                        continue

                print(f"현재까지 총 {len(data1)}개의 행을 수집했습니다.")

                # 페이지 이동 처리
                try:
                    pagination = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'paging')))
                    page_links = pagination.find_elements(By.XPATH, './/ul/li[@class=""]/a')
                    
                    if not page_links:  # 페이지 링크가 없으면 다음 버튼 확인
                        next_button = pagination.find_element(By.CLASS_NAME, 'icoBtn_next')
                        if next_button.get_attribute('disabled') is None:
                            print("다음 페이지 세트로 이동합니다.")
                            driver.execute_script("arguments[0].click();", next_button)
                            current_page = 1
                            time.sleep(3)
                            continue
                        else:
                            print("모든 페이지의 크롤링을 완료했습니다.")
                            break

                    max_pages = len(page_links) + 1
                    print(f"총 {max_pages}개의 페이지가 있습니다.")
                    
                    if current_page < max_pages:
                        next_page_link = page_links[current_page - 1]
                        print(f"{current_page + 1}페이지로 이동합니다.")
                        driver.execute_script("arguments[0].scrollIntoView(true);", next_page_link)
                        time.sleep(1)
                        driver.execute_script("arguments[0].click();", next_page_link)
                        current_page += 1
                        time.sleep(3)
                    else:
                        print("모든 페이지의 크롤링을 완료했습니다.")
                        break

                except Exception as e:
                    print(f"페이지 이동 중 오류 발생: {e}")
                    break

    except Exception as e:
        print(f"오류가 발생했습니다: {e}")

    df2 = pd.DataFrame(data1, columns=["판매구분", "판매사", "분류", "상품명", "판매기간", "요약서", "방법서", "약관"])
    return df2, driver

In [4]:
from webdriver_manager.chrome import ChromeDriverManager
# 메인 실행 부분
chrome_options = Options()
chrome_options.add_argument("--headless")  # 헤드리스 모드
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# ChromeDriverManager가 자동으로 알맞은 버전의 드라이버를 설치합니다
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

try:
    df1, driver = scrape_shinhan_life_products_on_sale(driver)
    df2, driver = scrape_shinhan_life_products_discontinued(driver)
    
    # 두 데이터프레임 합치기
    combined_df = pd.concat([df1, df2], ignore_index=True)
    # combined_df = df1
    
    # 합친 데이터프레임을 엑셀로 저장
    output_file = "shinhan_life_products_combined.xlsx"
    combined_df.to_excel(output_file, index=False)
    
    print(f"데이터를 {output_file} 파일로 저장했습니다.")
    print(f"총 {len(combined_df)} 개의 행이 저장되었습니다.")
    print(f"판매 중 상품: 총 {len(df1)} 개의 행")
    print(f"판매 중지 상품: 총 {len(df2)} 개의 행")
    
    # 판매구분별 행 수 출력
    print("\n판매구분별 행 수:")
    print(combined_df['판매구분'].value_counts())
    
    # 판매사별 행 수 출력
    print("\n판매사별 행 수:")
    print(combined_df['판매사'].value_counts())
# try:
#     df1, driver = scrape_shinhan_life_products_on_sale(driver)
    
#     # df1을 엑셀로 저장
#     output_file = "shinhan_life_products.xlsx"
#     df1.to_excel(output_file, index=False)
    
#     print(f"데이터를 {output_file} 파일로 저장했습니다.")
#     print(f"총 {len(df1)} 개의 행이 저장되었습니다.")
    
#     # 판매구분별 행 수 출력
#     print("\n판매구분별 행 수:")
#     print(df1['판매구분'].value_counts())
    
#     # 판매사별 행 수 출력
#     print("\n판매사별 행 수:")
#     print(df1['판매사'].value_counts())

finally:
    driver.quit()
    print("브라우저를 종료했습니다.")

공시실 상품공시 판매중 웹사이트에 접속했습니다.

현재 페이지: 1
총 51개의 행을 찾았습니다.
현재까지 총 254개의 행을 수집했습니다.
총 3개의 페이지가 있습니다.
2페이지로 이동합니다.

현재 페이지: 2
총 51개의 행을 찾았습니다.
현재까지 총 570개의 행을 수집했습니다.
총 3개의 페이지가 있습니다.
3페이지로 이동합니다.

현재 페이지: 3
총 2개의 행을 찾았습니다.
현재까지 총 600개의 행을 수집했습니다.
총 3개의 페이지가 있습니다.
모든 페이지의 크롤링을 완료했습니다.

공시실 상품공시 판매중지 웹사이트에 접속했습니다.
총 3개의 탭을 찾았습니다.

신한라이프 탭 처리 중...

현재 페이지: 1
총 51개의 행을 찾았습니다.
현재까지 총 197개의 행을 수집했습니다.
총 6개의 페이지가 있습니다.
2페이지로 이동합니다.

현재 페이지: 2
총 51개의 행을 찾았습니다.
현재까지 총 371개의 행을 수집했습니다.
총 6개의 페이지가 있습니다.
3페이지로 이동합니다.

현재 페이지: 3
총 51개의 행을 찾았습니다.
현재까지 총 530개의 행을 수집했습니다.
총 6개의 페이지가 있습니다.
4페이지로 이동합니다.

현재 페이지: 4
총 51개의 행을 찾았습니다.
현재까지 총 738개의 행을 수집했습니다.
총 6개의 페이지가 있습니다.
5페이지로 이동합니다.

현재 페이지: 5
총 51개의 행을 찾았습니다.
현재까지 총 874개의 행을 수집했습니다.
총 6개의 페이지가 있습니다.
6페이지로 이동합니다.

현재 페이지: 6
총 3개의 행을 찾았습니다.
현재까지 총 882개의 행을 수집했습니다.
총 6개의 페이지가 있습니다.
모든 페이지의 크롤링을 완료했습니다.

(구)신한생명 탭 처리 중...

현재 페이지: 1
총 51개의 행을 찾았습니다.
현재까지 총 1074개의 행을 수집했습니다.
총 10개의 페이지가 있습니다.
2페이지로 이동합니다.

현재 페이지: 2
총 51개의 행을 찾았습니다.
현재까지 총 1274개의 행을 수집했습니다.
총 1